In [ ]:
!unzip /content/dataset.zip

In [ ]:
import pandas as pd,string

In [ ]:
imb_df=pd.read_csv('/content/imbalanced_data.csv')

In [ ]:
imb_df.head()

In [ ]:
imb_df['label'].value_counts()

In [ ]:
import seaborn as sns

In [ ]:
sns.countplot(x='label',data=imb_df)

In [ ]:
imb_df.isna().sum()

In [ ]:
imb_df.drop('id',axis=1,inplace=True)

In [ ]:
raw_df=pd.read_csv('/content/raw_data.csv')

In [ ]:
raw_df.head()

In [ ]:
raw_df.drop(['Unnamed: 0','count','hate_speech','offensive_language','neither'],axis=1,inplace =True)

In [ ]:
raw_df['class'].unique()

In [ ]:
sns.countplot(x='class',data=raw_df)

In [ ]:
raw_df[raw_df['class']==0]['class']=1

In [ ]:
raw_df['class'].replace({0:1},inplace=True)

In [ ]:
raw_df["class"].replace({2:0}, inplace = True)

In [ ]:
raw_df.rename(columns={'class':'label'},inplace=True)

In [ ]:
frame=[imb_df,raw_df]

In [ ]:
df=pd.concat(frame)

In [ ]:
import re,nltk

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
stemmer=nltk.SnowballStemmer('english')
stopword=set(stopwords.words('english'))

In [ ]:
def data_cleaning_reg(words):
    words = str(words).lower()
    words = re.sub('\[.*?\]','',words)
    words = re.sub('https?://\S+|www\.\S+', '', words)
    words = re.sub('<.*?>+', '', words)
    words = re.sub('[%s]' % re.escape(string.punctuation), '', words)
    words = re.sub('\n', '', words)
    words = re.sub('\w*\d\w*', '', words)
    words = [word for word in words.split(' ') if word not in stopword]
    words=" ".join(words)
    words = [stemmer.stem(words) for word in words.split(' ')]
    words=" ".join(words)

    return words


In [ ]:
!pip install neattext

In [ ]:
import neattext as nt
from neattext.functions import clean_text, remove_punctuations, remove_stopwords, remove_special_characters, remove_urls, remove_emojis, remove_numbers

def data_cleaning_neat(text):
    # Convert to lowercase
    text = str(text).lower()

    # Remove URLs
    text = remove_urls(text)

    # Remove punctuation
    text = remove_punctuations(text)

    # Remove special characters
    text = remove_special_characters(text)

    # Remove numbers
    text = remove_numbers(text)
    # Remove stopwords
    text = remove_stopwords(text)

    # Remove emojis
    text = remove_emojis(text)

    # Remove extra whitespaces
    text = " ".join(text.split())

    return text


In [ ]:
df['tweet'][1]

In [ ]:
df['tweet']=df['tweet'].apply(data_cleaning_reg)

In [ ]:
df['tweet'][1]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x=df['tweet']
y=df['label']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42)

In [ ]:
!pip install keras

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
max_words=50000
max_len=300
tokenizer=Tokenizer(num_words=max_words)
x_train_clean = [data_cleaning_reg(t) for t in x_train]
tokenizer.fit_on_texts(x_train_clean)
sequences=tokenizer.texts_to_sequences(x_train_clean)
sequences_matrix=pad_sequences(sequences,maxlen=max_len)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-1029877752.py", line 4, in <cell line: 0>
    x_train_clean = [data_cleaning_reg(t) for t in x_train]
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1029877752.py", line 4, in <listcomp>
    x_train_clean = [data_cleaning_reg(t) for t in x_train]
                     ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-692528076.py", line 11, in data_cleaning_reg
    words = [stemmer.stem(words) for word in words.split(' ')]
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-692528076.py", line 11, in <listcomp>
    words = [stemmer.stem(words) for word in words.split(' ')]
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nltk/stem/snowball.py", line None, in stem

In [ ]:
sequences_matrix

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D
from keras.optimizers import RMSprop

In [ ]:
model = Sequential()
model.add(Embedding(max_words,100,input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
test_sequences = tokenizer.texts_to_sequences(x_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,y_test)

130/444 ━━━━━━━━━━━━━━━━━━━━ 51s 163ms/step - accuracy: 0.5878 - loss: 0.6921

In [ ]:
lstm_prediction = model.predict(test_sequences_matrix)

In [ ]:
res = []
for prediction in lstm_prediction:
    if prediction[0] < 0.5:
        res.append(0)
    else:
        res.append(1)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
print(confusion_matrix(y_test,res))

In [ ]:
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
model.save("model.h5")

In [ ]:
import keras

In [ ]:
load_model=keras.models.load_model("model.h5")
with open('tokenizer.pickle', 'rb') as handle:
    load_tokenizer = pickle.load(handle)

In [ ]:
test = 'i love this movie'

def clean_text(text):
    print(text)
    text = str(text).lower()
    text = re.sub('\[.*?\]','',text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    print(text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

test=[clean_text(test)]
print(test)

seq = load_tokenizer.texts_to_sequences(test)
padded = pad_sequences(seq, maxlen=300)
print(seq)

pred = load_model.predict(padded)

print("pred", pred)
if pred<0.5:
    print("no hate")
else:
    print("hate and abusive")


In [ ]:
print(len(x_train),len(y_train))
print(len(x_test),len(y_test))